# Quantitative Momentum Strategy

"Momentum investing' means investing in stocks that have increased in price the most.

In this project we select 50 stocks with highes price momentum and then we calculate recommended trades for an equal-weight portfolio of these 50 stocks.

# Library imports

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
#calculates percentile scores and basic statistics
import xlsxwriter

# Importing list of stocks

In [2]:
stocks = pd.read_csv(r"C:\Users\nik10\iCloudDrive\Documents\Python Trading Robot\Project 1 - Equal Weight Index Fund\starter_files\sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN

'Tpk_059b97af715d417d9f49f50b51b1c448'

# Making Our First API Call
1st Version of 'momentum screener'

We need to get one-year price returns for each stock

IEX Cloud Docs: https://iexcloud.io/docs/api/ 

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


# Parsing Our API Call
Obtaining the right data we need

In [4]:
data['year1ChangePercent']

0.2168294510086513

In [5]:
symbol = 'ETFC'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data['year1ChangePercent']
data

{'companyName': 'E*TRADE Financial Corp.',
 'marketcap': 11066143373,
 'week52high': 58.3,
 'week52low': 26.92,
 'week52highSplitAdjustOnly': None,
 'week52lowSplitAdjustOnly': None,
 'week52change': 39.5705,
 'sharesOutstanding': 227037279,
 'float': 224585924,
 'avg10Volume': 3776831,
 'avg30Volume': 2652162,
 'day200MovingAvg': 46.88,
 'day50MovingAvg': 54.46,
 'employees': 4176,
 'ttmEPS': 3.5423,
 'ttmDividendRate': 0.57,
 'dividendYield': None,
 'peRatio': None,
 'beta': None,
 'maxChangePercent': None,
 'year5ChangePercent': None,
 'year2ChangePercent': None,
 'year1ChangePercent': None,
 'ytdChangePercent': None,
 'month6ChangePercent': None,
 'month3ChangePercent': None,
 'month1ChangePercent': None,
 'day30ChangePercent': None,
 'day5ChangePercent': None}

# Executing A Batch API Call & Building the dataframe

In [6]:

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

    
my_columns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy'] 

print(symbol_strings)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

Blank DataFrame + add the data

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
#    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    #use split method to undo join
    for symbol in symbol_string.split(','):
        
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True
    )
        
final_dataframe

,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,A,172.82,0.743887,N/A
1,AAL,20.32,0.587508,N/A
2,AAP,211.42,0.344505,N/A
3,AAPL,155.49,0.216882,N/A
4,ABBV,121.77,0.327288,N/A
...,...,...,...,...
500,YUM,137.66,0.450943,N/A
501,ZBH,152.76,0.075041,N/A
502,ZBRA,594.20,1.103511,N/A
503,ZION,58.40,0.801538,N/A


# Removing Low-Momentum Stocks

We want to only keep the top 50 stocks from the S&P500 that had the largest momentum
We need to remove all other stocks that are bellow tthe threshold.
Sort stocks' one-year price retrn and drop outside of top-50

In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,82.77,2.377844,N/A
1,272,KSS,60.87,2.123602,N/A
2,441,TPR,44.68,1.961601,N/A
3,344,NUE,129.12,1.77195,N/A
4,129,DFS,135.33,1.719824,N/A
5,148,DVN,29.00,1.703853,N/A
6,106,COF,171.22,1.691183,N/A
7,23,ALB,235.70,1.599553,N/A
8,253,IVZ,25.05,1.541064,N/A
9,453,UAA,25.04,1.475784,N/A


# Calculating the Number of Shares to Buy

In [9]:

def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        float(portfolio_size)
    #forces portfolio_size variable to become float
#    print (val)
    except ValueError:
        print('Only numerical characters are allowed! \nPlease try again:')
        portfolio_size = input('Enter the value of your portfolio:')

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000003
10000003


In [10]:
#portfolio_size
#output is a string
position_size = float(portfolio_size)/len(final_dataframe.index)
#   print(position_size)
#shows how much money should be invested in each stock

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe
#    print(math.floor(number_of_apple_shares)) #rounds down each position size
#2:14:04

D:\Users\nik10\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,82.77,2.377844,2416
1,272,KSS,60.87,2.123602,3285
2,441,TPR,44.68,1.961601,4476
3,344,NUE,129.12,1.77195,1548
4,129,DFS,135.33,1.719824,1477
5,148,DVN,29.00,1.703853,6896
6,106,COF,171.22,1.691183,1168
7,23,ALB,235.70,1.599553,848
8,253,IVZ,25.05,1.541064,7984
9,453,UAA,25.04,1.475784,7987


# Building a Better Momentum Strategy
We need to differentitate between qualities of stocks:

Categorising as follows:
High-quality stocks that show consistent outperformance over long periods of time.  

Low quality stocks that may not show much momentum but then surge upwards.

Identifying high-quality momentum we select stocks from the highest percentiles of:

* 1-month price return
* 3-month price return
* 6-month price return
* 1-year price return

Abbreviations: `hqm` - `High-quality momentum` stocks

In [11]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
hqm_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
#    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    #use split method to undo join
    for symbol in symbol_string.split(','):
        
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )
        


# Calculating Momentum Percentiles

In [12]:
#some entries have null output which can crash the code - removing nulls for empty symbols
hqm_dataframe=hqm_dataframe.dropna()

#array to run the loop to calculate percentiles
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row,  percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
                                                                                
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,178.40,N/A,0.752074,84.231537,0.410027,94.211577,0.298399,96.207585,0.119202,95.209581,N/A
1,AAL,20.08,N/A,0.594359,71.856287,-0.078146,9.580838,-0.158475,5.189621,-0.099716,4.590818,N/A
2,AAP,215.66,N/A,0.356615,44.311377,0.270329,78.243513,0.074101,66.067864,-0.033691,20.359281,N/A
3,AAPL,154.51,N/A,0.2187,26.946108,0.197962,65.668663,0.197383,89.221557,0.009679,45.10978,N/A
4,ABBV,124.06,N/A,0.32708,41.117764,0.153076,55.688623,0.040852,56.087824,0.010027,45.309381,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,136.49,N/A,0.440931,57.285429,0.305525,82.634731,0.135134,79.640719,0.100264,92.41517,N/A
501,ZBH,150.87,N/A,0.075812,12.37525,-0.099931,7.784431,-0.130192,9.580838,-0.092759,5.788423,N/A
502,ZBRA,587.92,N/A,1.109459,94.610778,0.166366,59.680639,0.17543,86.626747,0.055747,73.253493,N/A
503,ZION,55.90,N/A,0.807921,86.826347,0.026396,25.349301,-0.049692,26.347305,0.089014,89.221557,N/A


# Calculating the HQM score
`HQM Score` is High-quality momentum score that will filter stocks - calculated by taking the mean of the 4 percentile scores across 4 different timeframes

In [13]:
from statistics import mean
# mean([])



for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
            momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,178.40,N/A,0.752074,84.231537,0.410027,94.211577,0.298399,96.207585,0.119202,95.209581,92.46507
1,AAL,20.08,N/A,0.594359,71.856287,-0.078146,9.580838,-0.158475,5.189621,-0.099716,4.590818,22.804391
2,AAP,215.66,N/A,0.356615,44.311377,0.270329,78.243513,0.074101,66.067864,-0.033691,20.359281,52.245509
3,AAPL,154.51,N/A,0.2187,26.946108,0.197962,65.668663,0.197383,89.221557,0.009679,45.10978,56.736527
4,ABBV,124.06,N/A,0.32708,41.117764,0.153076,55.688623,0.040852,56.087824,0.010027,45.309381,49.550898
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,136.49,N/A,0.440931,57.285429,0.305525,82.634731,0.135134,79.640719,0.100264,92.41517,77.994012
501,ZBH,150.87,N/A,0.075812,12.37525,-0.099931,7.784431,-0.130192,9.580838,-0.092759,5.788423,8.882236
502,ZBRA,587.92,N/A,1.109459,94.610778,0.166366,59.680639,0.17543,86.626747,0.055747,73.253493,78.542914
503,ZION,55.90,N/A,0.807921,86.826347,0.026396,25.349301,-0.049692,26.347305,0.089014,89.221557,56.936128


# Selecting the 50 best momentum stocks
Sort DataFrame on the `HQM Score` and pick highest 50 entries

In [14]:
hqm_dataframe.sort_values(by='HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,230.90,N/A,1.610947,98.602794,0.504844,98.003992,0.452418,99.001996,0.205719,99.201597,98.702595
1,FTNT,306.77,N/A,1.344372,97.40519,0.837498,99.800399,0.481058,99.600798,0.117175,94.810379,97.904192
2,IT,302.87,N/A,1.396315,97.60479,0.680957,99.401198,0.305793,96.407186,0.163149,98.003992,97.854291
3,NUE,125.63,N/A,1.791003,99.401198,1.003562,100.0,0.179885,87.42515,0.272625,100.0,96.706587
4,NVDA,223.91,N/A,0.759001,84.431138,0.554561,99.001996,0.486178,99.800399,0.128327,96.407186,94.91018
5,LB,82.43,N/A,2.297869,100.0,0.816386,99.600798,0.223774,91.616766,0.080724,87.225549,94.610778
6,WST,465.61,N/A,0.675599,79.241517,0.634502,99.201597,0.346585,97.60479,0.176129,98.203593,93.562874
7,CARR,59.00,N/A,0.915243,90.419162,0.528466,98.602794,0.280317,95.209581,0.079169,86.626747,92.714571
8,A,178.40,N/A,0.752074,84.231537,0.410027,94.211577,0.298399,96.207585,0.119202,95.209581,92.46507
9,LLY,267.46,N/A,0.856976,88.622754,0.347121,88.023952,0.354903,97.804391,0.101308,92.61477,91.766467


# Calculating the number of shares to buy


In [15]:
portfolio_input()

position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in range(0, len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

Enter the value of your portfolio:1000000


D:\Users\nik10\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,230.90,86,1.610947,98.602794,0.504844,98.003992,0.452418,99.001996,0.205719,99.201597,98.702595
1,FTNT,306.77,65,1.344372,97.40519,0.837498,99.800399,0.481058,99.600798,0.117175,94.810379,97.904192
2,IT,302.87,66,1.396315,97.60479,0.680957,99.401198,0.305793,96.407186,0.163149,98.003992,97.854291
3,NUE,125.63,159,1.791003,99.401198,1.003562,100.0,0.179885,87.42515,0.272625,100.0,96.706587
4,NVDA,223.91,89,0.759001,84.431138,0.554561,99.001996,0.486178,99.800399,0.128327,96.407186,94.91018
5,LB,82.43,242,2.297869,100.0,0.816386,99.600798,0.223774,91.616766,0.080724,87.225549,94.610778
6,WST,465.61,42,0.675599,79.241517,0.634502,99.201597,0.346585,97.60479,0.176129,98.203593,93.562874
7,CARR,59.00,338,0.915243,90.419162,0.528466,98.602794,0.280317,95.209581,0.079169,86.626747,92.714571
8,A,178.40,112,0.752074,84.231537,0.410027,94.211577,0.298399,96.207585,0.119202,95.209581,92.46507
9,LLY,267.46,74,0.856976,88.622754,0.347121,88.023952,0.354903,97.804391,0.101308,92.61477,91.766467


# Formatting excel output

In [21]:
writer = pd.ExcelWriter('Momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

# Creating format for .xlsx file

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)
percent_format = writer.book.add_format(
    {
        'num_format' : '0.0%',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

Applying the Formats to the Columns of Our .xlsx File
Use set_column method applied to the writer.book object to apply formats to specific columns of our spreadsheets.

In [27]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price',dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['One-Year Price Return', dollar_format],
    'E': ['One-Year Return Percentile', percent_format],
    'F': ['Six-Month Price Return', dollar_format],
    'G': ['Six-Month Return Percentile', percent_format],
    'H': ['Three-Month Price Return', dollar_format],
    'I': ['Three-Month Return Percentile', percent_format],
    'J': ['One-Month Price Return', dollar_format],
    'K': ['One-Month Return Percentile', percent_format],
    'L': ['HQM Score', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0], column_formats[column][1])

In [28]:
writer.save()  